<a href="https://colab.research.google.com/github/open-mmlab/mmselfsup/blob/master/demo/mmselfsup_colab_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMSelfSup Tutorial
In this tutorial, we will introduce the following content:

- How to install MMSelfSup
- How to train algorithms in MMSelfSup
- How to train downstream tasks

If you have any other questions, welcome to report issues.

## How to install MMSelfSup

Before using MMSelfSup, we need to prepare the environment with the following steps:

1. Install Python, CUDA, C/C++ compiler and git
2. Install PyTorch (CUDA version)
3. Install dependent codebase (mmengine, mmcv, mmcls)
4. Clone mmselfsup source code from GitHub and install it

Because this tutorial is on Google Colab and all necessary packages have been installed, we can skip the first two steps.

In [1]:
# Check nvcc version
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [2]:
# Check GCC version
!gcc --version

gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
# Check PyTorch installation
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())

1.12.1+cu113
True


### Install MMEngine and MMCV

MMCV is the basic package of all OpenMMLab packages. We have pre-built wheels on Linux, so we can download and install them directly.

Please pay attention to PyTorch and CUDA versions to match the wheel.

In the above steps, we have checked the version of PyTorch and CUDA, and they are 1.10.2 and 11.3 respectively, so we need to choose the corresponding wheel.

In addition, we can also install the full version of mmcv (mmcv-full). It includes full features and various CUDA ops out of the box, but needs a longer time to build.

MIM is recommended: https://github.com/open-mmlab/mim

In [4]:
!pip3 install openmim
!pip install -U openmim
!mim install 'mmengine==0.1.0' 'mmcv>=2.0.0rc1'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49 kB 3.1 MB/s 
     |████████████████████████████████| 235 kB 9.7 MB/s 
     |████████████████████████████████| 51 kB 8.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
     |████████████████████████████████| 280 kB 5.2 MB/s 
     |████████████████████████████████| 40.4 MB 12.3 MB/s 
     |████████████████████████████████| 190 kB 64.2 MB/s 


In [5]:
# check mmengine install
!python -c 'from mmengine.utils.dl_utils import collect_env;print(collect_env())'

# check mmcv install
import mmcv
print(mmcv.__version__)

OrderedDict([('sys.platform', 'linux'), ('Python', '3.7.13 (default, Apr 24 2022, 01:04:09) [GCC 7.5.0]'), ('CUDA available', True), ('numpy_random_seed', 2147483648), ('GPU 0', 'Tesla T4'), ('CUDA_HOME', '/usr/local/cuda'), ('NVCC', 'Cuda compilation tools, release 11.1, V11.1.105'), ('GCC', 'x86_64-linux-gnu-gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0'), ('PyTorch', '1.12.1+cu113'), ('PyTorch compiling details', 'PyTorch built with:\n  - GCC 9.3\n  - C++ Version: 201402\n  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - LAPACK is enabled (usually provided by MKL)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 11.3\n  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,c

Besides, you can also use pip to install the packages, but you are supposed to check the pytorch and cuda version manually. The example command is provided below, but you need to modify it according to your PyTorch and CUDA version.

### Clone and install mmselfsup

In [6]:
%cd /content
# Clone MMSelfSup repository
!git clone https://github.com/open-mmlab/mmselfsup.git
%cd mmselfsup/

# Install MMSelfSup from source
!git checkout dev-1.x
!pip install -e . 

/content
Cloning into 'mmselfsup'...
remote: Enumerating objects: 6421, done.
remote: Counting objects: 100% (279/279), done.
remote: Compressing objects: 100% (192/192), done.
remote: Total 6421 (delta 123), reused 194 (delta 86), pack-reused 6142
Receiving objects: 100% (6421/6421), 2.75 MiB | 12.05 MiB/s, done.
Resolving deltas: 100% (4028/4028), done.
/content/mmselfsup
Branch 'dev-1.x' set up to track remote branch 'dev-1.x' from 'origin'.
Switched to a new branch 'dev-1.x'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmselfsup
     |████████████████████████████████| 557 kB 5.1 MB/s 
  Running setup.py develop for mmselfsup


In [7]:
# Check MMClassification installation
import mmcls
print(mmcls.__version__)

1.0.0rc0


In [8]:
# Check MMSelfSup installation
import mmselfsup
print(mmselfsup.__version__)

1.0.0rc1


## Example to start a self-supervised task

Before you start training, you need to prepare your dataset, please check [prepare_data.md](https://github.com/open-mmlab/mmselfsup/blob/master/docs/en/prepare_data.md) file carefully.

**Note**: As we follow the original algorithms to implement our codes, so many algorithms are supposed to run on distributed mode, they are not supported on 1 GPU training officially. You can check it [here](https://github.com/open-mmlab/mmselfsup/blob/master/tools/train.py#L120).


Here we provide a example and download a small dataset to display the demo.

### Prerapre data

In [9]:
!mkdir data
!wget https://download.openmmlab.com/mmselfsup/data/imagenet_examples.zip
!unzip -q imagenet_examples.zip -d ./data/

--2022-09-01 16:09:11--  https://download.openmmlab.com/mmselfsup/data/imagenet_examples.zip
Resolving download.openmmlab.com (download.openmmlab.com)... 47.89.140.71
Connecting to download.openmmlab.com (download.openmmlab.com)|47.89.140.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155496559 (148M) [application/zip]
Saving to: ‘imagenet_examples.zip’

imagenet_examples.z 100%[===================>] 148.29M  8.65MB/s    in 17s     

2022-09-01 16:09:29 (8.63 MB/s) - ‘imagenet_examples.zip’ saved [155496559/155496559]



In [10]:
# Check data directory
!apt-get install tree
!tree -d ./data

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (113 kB/s)
Selecting previously unselected package tree.
(Reading database ... 155676 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
./data
└── imagenet
    ├── meta
    └── train
        └── n01440764

4 directories


### Create a new config file
To reuse the common parts of different config files, we support inheriting multiple base config files. For example, to train `relative_loc` algorithm, the new config file can create the model's basic structure by inheriting `configs/_base_/models/relative-loc.py`.

In [11]:
%%writefile configs/selfsup/relative_loc/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab.py
_base_ = [
    '../_base_/models/relative-loc.py',
    '../_base_/datasets/imagenet_relative-loc.py',
    '../_base_/schedules/sgd_steplr-200e_in1k.py',
    '../_base_/default_runtime.py',
]

default_hooks = dict(logger=dict(type='LoggerHook', interval=10))

# optimizer wrapper
optimizer = dict(type='SGD', lr=0.2, momentum=0.9, weight_decay=1e-4)
optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=optimizer,
    paramwise_cfg=dict(custom_keys={
        'neck': dict(decay_mult=5.0),
        'head': dict(decay_mult=5.0)
    }))

# learning rate scheduler
param_scheduler = [dict(type='MultiStepLR', by_epoch=True, milestones=[1, 2])]

# runtime settings
# pre-train for 70 epochs
train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=70)
# the max_keep_ckpts controls the max number of ckpt file in your work_dirs
# if it is 3, when CheckpointHook (in mmcv) saves the 4th ckpt
# it will remove the oldest one to keep the number of total ckpts as 3
default_hooks = dict(
    checkpoint=dict(type='CheckpointHook', interval=1, max_keep_ckpts=3))


Writing configs/selfsup/relative_loc/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab.py


### Read the config file and modify config

We can modify the loaded config file.

In [12]:
# Load the basic config file
from mmengine.config import Config
cfg = Config.fromfile('configs/selfsup/relative_loc/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab.py')

# Specify the data settings
cfg.train_dataloader.batch_size = 8
cfg.train_dataloader.num_workers = 2

# Specify the optimizer
cfg.optimizer = dict(type='SGD', lr=0.005, momentum=0.9, weight_decay=0.0001)
cfg.optim_wrapper.clip_grad = None

# Specify the learning rate scheduler
cfg.param_scheduler = [dict(type='MultiStepLR', by_epoch=True, milestones=[1, 2])]

# Modify runtime setting
cfg.train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=2)

# Specify the work directory
cfg.work_dir = './work_dirs/selfsup/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab'

# Output logs for every 10 iterations
cfg.default_hooks.logger.interval = 10
# Set the random seed and enable the deterministic option of cuDNN
# to keep the results' reproducible.
cfg.randomness = dict(seed=0, deterministic=True)

### Start self-supervised pre-train task

In [13]:
import os
import torch

if torch.cuda.is_available():
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':16:8'

In [14]:
from mmengine.config import Config, DictAction
from mmengine.runner import Runner

from mmselfsup.utils import register_all_modules

# register all modules in mmselfsup into the registries
# do not init the default scope here because it will be init in the runner
register_all_modules(init_default_scope=False)

# build the runner from config
runner = Runner.from_cfg(cfg)

# start training
runner.train()

09/01 16:09:40 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.7.13 (default, Apr 24 2022, 01:04:09) [GCC 7.5.0]
    CUDA available: True
    numpy_random_seed: 0
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.1, V11.1.105
    GCC: x86_64-linux-gnu-gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
    PyTorch: 1.12.1+cu113
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;

RelativeLoc(
  (data_preprocessor): RelativeLocDataPreprocessor()
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(in

## Example to start a downstream task


### Extract backbone weights from pre-train model

In [15]:
!python tools/model_converters/extract_backbone_weights.py \
  work_dirs/selfsup/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab/epoch_2.pth \
  work_dirs/selfsup/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab/relative-loc_backbone-weights.pth

### Prepare config file

Here we create a new config file for demo dataset, actually we provided various config files in directory `configs/benchmarks`.

In [16]:
# Load the basic config file
from mmengine.config import Config
benchmark_cfg = Config.fromfile('configs/benchmarks/classification/imagenet/resnet50_linear-8xb32-steplr-100e_in1k.py')

# Modify the model
checkpoint_file = 'work_dirs/selfsup/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab/relative-loc_backbone-weights.pth'
# Or directly using pre-train model provided by us
# checkpoint_file = 'https://download.openmmlab.com/mmselfsup/moco/mocov2_resnet50_8xb32-coslr-200e_in1k_20220225-89e03af4.pth'

benchmark_cfg.model.backbone.frozen_stages=4
benchmark_cfg.model.backbone.init_cfg = dict(type='Pretrained', checkpoint=checkpoint_file)

# As the imagenet_examples dataset folder doesn't have val dataset
# Modify the path and meta files of validation dataset
benchmark_cfg.val_dataloader.dataset.data_prefix = 'train'
benchmark_cfg.val_dataloader.dataset.ann_file = 'meta/train.txt'

# Specify the learning rate scheduler
benchmark_cfg.param_scheduler = [dict(type='MultiStepLR', by_epoch=True, milestones=[1, 2])]

# Output logs for every 10 iterations
benchmark_cfg.default_hooks.logger.interval = 10

# Modify runtime settings for demo
benchmark_cfg.train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=2)


# Specify the work directory
benchmark_cfg.work_dir = './work_dirs/benchmarks/classification/imagenet/resnet50_8xb32-steplr-100e_in1k_colab'

# Set the random seed and enable the deterministic option of cuDNN
# to keep the results' reproducible.
benchmark_cfg.randomness = dict(seed=0, deterministic=True)

### Load extracted backbone weights to start a downstream task

In [17]:
from mmengine.config import Config, DictAction
from mmengine.runner import Runner

from mmselfsup.utils import register_all_modules

# register all modules in mmselfsup into the registries
# do not init the default scope here because it will be init in the runner
register_all_modules(init_default_scope=False)

# build the runner from config
runner = Runner.from_cfg(benchmark_cfg)

# start training
runner.train()

09/01 16:11:10 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.7.13 (default, Apr 24 2022, 01:04:09) [GCC 7.5.0]
    CUDA available: True
    numpy_random_seed: 0
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.1, V11.1.105
    GCC: x86_64-linux-gnu-gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
    PyTorch: 1.12.1+cu113
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


09/01 16:11:24 - mmengine - INFO - load model from: work_dirs/selfsup/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab/relative-loc_backbone-weights.pth
09/01 16:11:24 - mmengine - INFO - local loads checkpoint from path: work_dirs/selfsup/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab/relative-loc_backbone-weights.pth
09/01 16:11:24 - mmengine - INFO - 
backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from work_dirs/selfsup/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab/relative-loc_backbone-weights.pth 
 
09/01 16:11:24 - mmengine - INFO - 
backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from work_dirs/selfsup/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab/relative-loc_backbone-weights.pth 
 
09/01 16:11:24 - mmengine - INFO - 
backbone.bn1.bias - torch.Size([64]): 
PretrainedInit: load from work_dirs/selfsup/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab/relative-loc_backbone-weights.pth 
 
09/01 16:11:24 - mmengine - INFO - 
backbo

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


09/01 16:11:30 - mmengine - INFO - Epoch(train) [1][10/41]  lr: 3.0000e+01  eta: 0:00:35  time: 0.4955  data_time: 0.3703  memory: 1392  loss: 1.0352
09/01 16:11:32 - mmengine - INFO - Epoch(train) [1][20/41]  lr: 3.0000e+01  eta: 0:00:23  time: 0.2497  data_time: 0.1329  memory: 762  loss: 0.0000
09/01 16:11:35 - mmengine - INFO - Epoch(train) [1][30/41]  lr: 3.0000e+01  eta: 0:00:17  time: 0.2528  data_time: 0.1333  memory: 762  loss: 0.0000
09/01 16:11:37 - mmengine - INFO - Epoch(train) [1][40/41]  lr: 3.0000e+01  eta: 0:00:12  time: 0.2088  data_time: 0.0967  memory: 762  loss: 0.0000
09/01 16:11:37 - mmengine - INFO - Exp name: resnet50_linear-8xb32-steplr-100e_in1k_20220901_161110
09/01 16:11:41 - mmengine - INFO - Epoch(val) [1][10/41]    eta: 0:00:12  time: 0.4174  data_time: 0.2966  memory: 762  
09/01 16:11:43 - mmengine - INFO - Epoch(val) [1][20/41]    eta: 0:00:03  time: 0.1886  data_time: 0.0625  memory: 762  
09/01 16:11:45 - mmengine - INFO - Epoch(val) [1][30/41]    e

ImageClassifier(
  (data_preprocessor): ClsDataPreprocessor()
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplac

**Note: As the demo only has one class in dataset, the model collapsed and the results of loss and acc should be ignored.**